<a href="https://colab.research.google.com/github/Piroon1123/choikeonwoo/blob/main/AI_06_%EC%B5%9C%EA%B1%B4%EC%9A%B0_Section04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project4 - 대출 상환능력 예측하기

## **0. 파일 불러오기**

In [ ]:
# 0-1 구글 드라이브 연동

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 0-2 데이터 불러오기 --> 엑셀 데이터가 굉장히 커서 결측값이 많은 항목, 타 항목과 내용이 겹치는 항목은 뺐습니다.

import pandas as pd

df_url = '/content/drive/MyDrive/Colab Notebooks/prjfile/cutomerdt.csv'
df = pd.read_csv(df_url, delimiter=',',encoding='latin-1')
df

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,DAYS_BIRTH,FLAG_MOBIL,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,ORGANIZATION_TYPE
0,100002,1,Cash loans,M,N,Y,202500.0,406597.5,24700.5,351000.0,Secondary / secondary special,Single / not married,-9461,1,0,1.0,2,WEDNESDAY,10,0,0,Business Entity Type 3
1,100003,0,Cash loans,F,N,N,270000.0,1293502.5,35698.5,1129500.0,Higher education,Married,-16765,1,0,2.0,1,MONDAY,11,0,0,School
2,100004,0,Revolving loans,M,Y,Y,67500.0,135000.0,6750.0,135000.0,Secondary / secondary special,Single / not married,-19046,1,0,1.0,2,MONDAY,9,0,0,Government
3,100006,0,Cash loans,F,N,Y,135000.0,312682.5,29686.5,297000.0,Secondary / secondary special,Civil marriage,-19005,1,0,2.0,2,WEDNESDAY,17,0,0,Business Entity Type 3
4,100007,0,Cash loans,M,N,Y,121500.0,513000.0,21865.5,513000.0,Secondary / secondary special,Single / not married,-19932,1,0,1.0,2,THURSDAY,11,0,0,Religion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306168,454733,0,Cash loans,M,Y,Y,360000.0,1051294.5,34042.5,918000.0,Secondary / secondary special,Married,-11380,1,0,3.0,1,TUESDAY,9,0,0,Transport: type 2
306169,454734,0,Revolving loans,F,N,Y,67500.0,202500.0,10125.0,202500.0,Secondary / secondary special,Married,-20113,1,0,2.0,2,MONDAY,13,0,0,XNA
306170,454735,0,Cash loans,F,N,Y,220500.0,319500.0,34537.5,319500.0,Higher education,Single / not married,-18039,1,0,2.0,3,THURSDAY,13,0,0,Self-employed
306171,454736,0,Cash loans,M,N,N,180000.0,398016.0,36958.5,360000.0,Secondary / secondary special,Single / not married,-10424,1,0,1.0,1,MONDAY,14,0,0,Business Entity Type 3


1. SK_ID_ CURR : 회원 식별 아이디
2. TARGET : 대출상환능력 (0: 상환가능 , 1: 상환불가능)
3. NAME_CONTRACT_TYPE : 대출 유형
4. CODE_GENDER	: 성별
5. FLAG_OWN_CAR : 차량소유여부
6. FLAG_OWN_REALTY	: 부동산 소유 여부
7. AMT_INCOME_TOTAL	: 현재까지 총소득 (단위 : 달러$, 이하 동일)
8. AMT_CREDIT	: 현재까지 총 융자 
9. AMT_ANNUITY	: 총 연금수령액
10. AMT_GOODS_PRICE : 상품을 구매하는 데 지불한 대출금	
11. NAME_EDUCATION_TYPE	: 교육 정도
12. NAME_FAMILY_STATUS	: 결혼여부
13. DAYS_BIRTH	: 나이
14. FLAG_MOBIL	: 휴대폰 보유여부
15. FLAG_EMAIL	: 이메일 보유여부
16. CNT_FAM_MEMBERS	: 구성 가족수
17. REGION_RATING_CLIENT	: 주거지 유형 (1: 도심 2: 부도심 3: 외곽)
18. WEEKDAY_APPR_PROCESS_START	: 대출신청요일
19. HOUR_APPR_PROCESS_START	: 대출신청시간
20. REG_REGION_NOT_LIVE_REGION : 서류상 주거지 / 실제 주거지 불일치 여부
21. REG_REGION_NOT_WORK_REGION : 서류상 근무지 / 실제 근무지 불일치 여부
22. ORGANIZATION_TYPE : 근무처 유형

## **I. 데이터 전처리**

### 1. 표 다듬기

In [ ]:
# 결측치 없애기

pd.DataFrame(df.isnull().sum(), columns=['결측치 갯수'])

,결측치 갯수
SK_ID_CURR,0
TARGET,0
NAME_CONTRACT_TYPE,0
CODE_GENDER,0
FLAG_OWN_CAR,0
FLAG_OWN_REALTY,0
AMT_INCOME_TOTAL,0
AMT_CREDIT,0
AMT_ANNUITY,12
AMT_GOODS_PRICE,277


In [ ]:
df = df.dropna()

In [ ]:
pd.DataFrame(df.isnull().sum(), columns=['결측치 갯수'])

,결측치 갯수
SK_ID_CURR,0
TARGET,0
NAME_CONTRACT_TYPE,0
CODE_GENDER,0
FLAG_OWN_CAR,0
FLAG_OWN_REALTY,0
AMT_INCOME_TOTAL,0
AMT_CREDIT,0
AMT_ANNUITY,0
AMT_GOODS_PRICE,0


In [ ]:
df = df.reset_index()
df

,index,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,DAYS_BIRTH,FLAG_MOBIL,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,ORGANIZATION_TYPE
0,0,100002,1,Cash loans,M,N,Y,202500.0,406597.5,24700.5,351000.0,Secondary / secondary special,Single / not married,-9461,1,0,1.0,2,WEDNESDAY,10,0,0,Business Entity Type 3
1,1,100003,0,Cash loans,F,N,N,270000.0,1293502.5,35698.5,1129500.0,Higher education,Married,-16765,1,0,2.0,1,MONDAY,11,0,0,School
2,2,100004,0,Revolving loans,M,Y,Y,67500.0,135000.0,6750.0,135000.0,Secondary / secondary special,Single / not married,-19046,1,0,1.0,2,MONDAY,9,0,0,Government
3,3,100006,0,Cash loans,F,N,Y,135000.0,312682.5,29686.5,297000.0,Secondary / secondary special,Civil marriage,-19005,1,0,2.0,2,WEDNESDAY,17,0,0,Business Entity Type 3
4,4,100007,0,Cash loans,M,N,Y,121500.0,513000.0,21865.5,513000.0,Secondary / secondary special,Single / not married,-19932,1,0,1.0,2,THURSDAY,11,0,0,Religion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305879,306168,454733,0,Cash loans,M,Y,Y,360000.0,1051294.5,34042.5,918000.0,Secondary / secondary special,Married,-11380,1,0,3.0,1,TUESDAY,9,0,0,Transport: type 2
305880,306169,454734,0,Revolving loans,F,N,Y,67500.0,202500.0,10125.0,202500.0,Secondary / secondary special,Married,-20113,1,0,2.0,2,MONDAY,13,0,0,XNA
305881,306170,454735,0,Cash loans,F,N,Y,220500.0,319500.0,34537.5,319500.0,Higher education,Single / not married,-18039,1,0,2.0,3,THURSDAY,13,0,0,Self-employed
305882,306171,454736,0,Cash loans,M,N,N,180000.0,398016.0,36958.5,360000.0,Secondary / secondary special,Single / not married,-10424,1,0,1.0,1,MONDAY,14,0,0,Business Entity Type 3


In [ ]:
# 필요없는 column 제거 --> 인덱스와 ID는 고객을 식별하는 수단일 뿐, 대출상환여부와는 관계가 없으므로 제거

df = df.drop(columns=['index', 'SK_ID_CURR'])
df

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,DAYS_BIRTH,FLAG_MOBIL,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,ORGANIZATION_TYPE
0,1,Cash loans,M,N,Y,202500.0,406597.5,24700.5,351000.0,Secondary / secondary special,Single / not married,-9461,1,0,1.0,2,WEDNESDAY,10,0,0,Business Entity Type 3
1,0,Cash loans,F,N,N,270000.0,1293502.5,35698.5,1129500.0,Higher education,Married,-16765,1,0,2.0,1,MONDAY,11,0,0,School
2,0,Revolving loans,M,Y,Y,67500.0,135000.0,6750.0,135000.0,Secondary / secondary special,Single / not married,-19046,1,0,1.0,2,MONDAY,9,0,0,Government
3,0,Cash loans,F,N,Y,135000.0,312682.5,29686.5,297000.0,Secondary / secondary special,Civil marriage,-19005,1,0,2.0,2,WEDNESDAY,17,0,0,Business Entity Type 3
4,0,Cash loans,M,N,Y,121500.0,513000.0,21865.5,513000.0,Secondary / secondary special,Single / not married,-19932,1,0,1.0,2,THURSDAY,11,0,0,Religion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305879,0,Cash loans,M,Y,Y,360000.0,1051294.5,34042.5,918000.0,Secondary / secondary special,Married,-11380,1,0,3.0,1,TUESDAY,9,0,0,Transport: type 2
305880,0,Revolving loans,F,N,Y,67500.0,202500.0,10125.0,202500.0,Secondary / secondary special,Married,-20113,1,0,2.0,2,MONDAY,13,0,0,XNA
305881,0,Cash loans,F,N,Y,220500.0,319500.0,34537.5,319500.0,Higher education,Single / not married,-18039,1,0,2.0,3,THURSDAY,13,0,0,Self-employed
305882,0,Cash loans,M,N,N,180000.0,398016.0,36958.5,360000.0,Secondary / secondary special,Single / not married,-10424,1,0,1.0,1,MONDAY,14,0,0,Business Entity Type 3


In [ ]:
# 1-3 용어 간소화 및 개념 통일

# 교육 정도 -> 대학교 중퇴 = Secondary

df['NAME_EDUCATION_TYPE'] = df['NAME_EDUCATION_TYPE'].replace({'Secondary / secondary special' : 'Secondary'})
df['NAME_EDUCATION_TYPE'] = df['NAME_EDUCATION_TYPE'].replace({'Incomplete higher' : 'Secondary'})

In [ ]:
# 결혼 여부 -> 미혼 / 과부 = Single , 사실혼 / 별거 = Married 

df['NAME_FAMILY_STATUS'] = df['NAME_FAMILY_STATUS'].replace({'Single / not married' : 'Single'})
df['NAME_FAMILY_STATUS'] = df['NAME_FAMILY_STATUS'].replace({'Widow' : 'Single'})
df['NAME_FAMILY_STATUS'] = df['NAME_FAMILY_STATUS'].replace({'Civil marriage' : 'Married'})
df['NAME_FAMILY_STATUS'] = df['NAME_FAMILY_STATUS'].replace({'Separated' : 'Married'})

In [ ]:
# Industry 항목 전부 Industry로 통일

df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Industry: type 1' : 'Industry'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Industry: type 2' : 'Industry'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Industry: type 3' : 'Industry'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Industry: type 4' : 'Industry'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Industry: type 5' : 'Industry'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Industry: type 6' : 'Industry'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Industry: type 7' : 'Industry'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Industry: type 8' : 'Industry'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Industry: type 9' : 'Industry'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Industry: type 10' : 'Industry'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Industry: type 11' : 'Industry'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Industry: type 12' : 'Industry'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Industry: type 13' : 'Industry'})

In [ ]:
# Trade 항목 전부 Trade로 통일

df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Trade: type 1' : 'Trade'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Trade: type 2' : 'Trade'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Trade: type 3' : 'Trade'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Trade: type 4' : 'Trade'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Trade: type 5' : 'Trade'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Trade: type 6' : 'Trade'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Trade: type 7' : 'Trade'})

In [ ]:
# Transport 항목 전부 Transport로 통일

df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Transport: type 1' : 'Transport'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Transport: type 2' : 'Transport'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Transport: type 3' : 'Transport'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Transport: type 4' : 'Transport'})

In [ ]:
# Business 항목 전부 Business로 통일

df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Business Entity Type 1' : 'Business Entity'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Business Entity Type 2' : 'Business Entity'})
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace({'Business Entity Type 3' : 'Business Entity'})

In [ ]:
# DAYS_BIRTH 항목 나이로 변환

def toage(num):
  age = num * (-1) / 365
  age = int(age)
  return age

In [ ]:
df['DAYS_BIRTH'] = df['DAYS_BIRTH'].apply(toage)

In [ ]:
# CNT_FAM_MEMBERS 항목 정수로 변환

def toint(num):
  numm = int(num)
  return numm

In [ ]:
df['CNT_FAM_MEMBERS'] = df['CNT_FAM_MEMBERS'].apply(toint)

In [ ]:
df

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,DAYS_BIRTH,FLAG_MOBIL,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,ORGANIZATION_TYPE
0,1,Cash loans,M,N,Y,202500.0,406597.5,24700.5,351000.0,Secondary,Single,25,1,0,1,2,WEDNESDAY,10,0,0,Business Entity
1,0,Cash loans,F,N,N,270000.0,1293502.5,35698.5,1129500.0,Higher education,Married,45,1,0,2,1,MONDAY,11,0,0,School
2,0,Revolving loans,M,Y,Y,67500.0,135000.0,6750.0,135000.0,Secondary,Single,52,1,0,1,2,MONDAY,9,0,0,Government
3,0,Cash loans,F,N,Y,135000.0,312682.5,29686.5,297000.0,Secondary,Married,52,1,0,2,2,WEDNESDAY,17,0,0,Business Entity
4,0,Cash loans,M,N,Y,121500.0,513000.0,21865.5,513000.0,Secondary,Single,54,1,0,1,2,THURSDAY,11,0,0,Religion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305879,0,Cash loans,M,Y,Y,360000.0,1051294.5,34042.5,918000.0,Secondary,Married,31,1,0,3,1,TUESDAY,9,0,0,Transport
305880,0,Revolving loans,F,N,Y,67500.0,202500.0,10125.0,202500.0,Secondary,Married,55,1,0,2,2,MONDAY,13,0,0,XNA
305881,0,Cash loans,F,N,Y,220500.0,319500.0,34537.5,319500.0,Higher education,Single,49,1,0,2,3,THURSDAY,13,0,0,Self-employed
305882,0,Cash loans,M,N,N,180000.0,398016.0,36958.5,360000.0,Secondary,Single,28,1,0,1,1,MONDAY,14,0,0,Business Entity


### 2. Onehot Encoding

In [ ]:
!pip install category_encoders

     |████████████████████████████████| 82 kB 430 kB/s 


In [ ]:
from category_encoders import OrdinalEncoder

encoder = OrdinalEncoder()
df_encoded = encoder.fit_transform(df)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df_encoded

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,DAYS_BIRTH,FLAG_MOBIL,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,ORGANIZATION_TYPE
0,1,1,1,1,1,202500.0,406597.5,24700.5,351000.0,1,1,25,1,0,1,2,1,10,0,0,1
1,0,1,2,1,2,270000.0,1293502.5,35698.5,1129500.0,2,2,45,1,0,2,1,2,11,0,0,2
2,0,2,1,2,1,67500.0,135000.0,6750.0,135000.0,1,1,52,1,0,1,2,2,9,0,0,3
3,0,1,2,1,1,135000.0,312682.5,29686.5,297000.0,1,2,52,1,0,2,2,1,17,0,0,1
4,0,1,1,1,1,121500.0,513000.0,21865.5,513000.0,1,1,54,1,0,1,2,3,11,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305879,0,1,1,2,1,360000.0,1051294.5,34042.5,918000.0,1,2,31,1,0,3,1,7,9,0,0,10
305880,0,2,2,1,1,67500.0,202500.0,10125.0,202500.0,1,2,55,1,0,2,2,2,13,0,0,6
305881,0,1,2,1,1,220500.0,319500.0,34537.5,319500.0,2,1,49,1,0,2,3,3,13,0,0,9
305882,0,1,1,1,2,180000.0,398016.0,36958.5,360000.0,1,1,28,1,0,1,1,2,14,0,0,1


### 3. Train / Test

In [ ]:
# 훈련 / 테스트 셋 분리

from sklearn.model_selection import train_test_split

train, test = train_test_split(df_encoded, test_size=0.2, random_state=1)

train.shape, test.shape

((244707, 21), (61177, 21))

In [ ]:
# features 와 target 을 분리

target = 'TARGET'
features = df_encoded.drop(columns=[target]).columns

X_train = train[features]
X_test = test[features]

y_train = train[target]
y_test = test[target]

### 4. StandardScaler

In [ ]:
# 정규화 과정

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## **II. 모델링**

### 1. Baseline Model

In [ ]:
!pip install -U keras-tuner

     |████████████████████████████████| 98 kB 1.8 MB/s 


In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras
import kerastuner as kt
import IPython
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """


In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu')) # 노드 수 : 128개 
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # 이진분류 -> 노드 : 1 / 활성화 함수 : sigmoid / 손실함수 : binary_crossentropy

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

results = model.fit(X_train_scaled, y_train, epochs=5, batch_size = 64, validation_data=(X_test_scaled,y_test))

Epoch 1/5
3824/3824 [==============================] - 9s 2ms/step - loss: 0.2746 - accuracy: 0.9189 - val_loss: 0.2676 - val_accuracy: 0.9204
Epoch 2/5
3824/3824 [==============================] - 8s 2ms/step - loss: 0.2709 - accuracy: 0.9190 - val_loss: 0.2697 - val_accuracy: 0.9204
Epoch 3/5
3824/3824 [==============================] - 8s 2ms/step - loss: 0.2696 - accuracy: 0.9190 - val_loss: 0.2666 - val_accuracy: 0.9204
Epoch 4/5
3824/3824 [==============================] - 8s 2ms/step - loss: 0.2689 - accuracy: 0.9190 - val_loss: 0.2669 - val_accuracy: 0.9204
Epoch 5/5
3824/3824 [==============================] - 8s 2ms/step - loss: 0.2683 - accuracy: 0.9190 - val_loss: 0.2661 - val_accuracy: 0.9204


In [ ]:
model.evaluate(X_test_scaled,  y_test, verbose=2)

1912/1912 - 2s - loss: 0.2661 - accuracy: 0.9204 - 2s/epoch - 935us/step


[0.26606884598731995, 0.920378565788269]

### 2. GridSearchCV

In [ ]:
def model_builder(nodes=128, activation='relu'):

  model2 = Sequential()
  model2.add(Dense(nodes, activation=activation))
  model2.add(Dense(nodes, activation=activation))
  model2.add(Dense(1, activation='sigmoid'))

  model2.compile(optimizer='adam', 
                loss='binary_crossentropy', 
                metrics=['accuracy'])

  return model2

model2 = KerasClassifier(build_fn=model_builder, verbose=0)

# GridSearch
batch_size = [64, 128]
epochs = [5]
nodes = [16, 32, 64, 128]
activation = ['relu', 'sigmoid']
param_grid = dict(batch_size=batch_size, epochs=epochs, nodes=nodes, activation=activation)

# GridSearch CV를 만들기
grid = GridSearchCV(estimator=model2, param_grid=param_grid, cv=2, verbose=2, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

Fitting 2 folds for each of 16 candidates, totalling 32 fits


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed:  5.6min finished


In [ ]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

Best: 0.9189765751361847 using {'activation': 'relu', 'batch_size': 64, 'epochs': 5, 'nodes': 32}


In [ ]:
model22 = Sequential()
model22.add(Dense(16, activation='relu'))
model22.add(Dense(16, activation='relu'))
model22.add(Dense(1, activation='sigmoid'))

model22.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

results = model22.fit(X_train_scaled, y_train, epochs=5, batch_size = 64, validation_data=(X_test_scaled,y_test))

Epoch 1/5
3824/3824 [==============================] - 7s 2ms/step - loss: 0.2772 - accuracy: 0.9190 - val_loss: 0.2690 - val_accuracy: 0.9204
Epoch 2/5
3824/3824 [==============================] - 6s 2ms/step - loss: 0.2719 - accuracy: 0.9190 - val_loss: 0.2680 - val_accuracy: 0.9204
Epoch 3/5
3824/3824 [==============================] - 6s 2ms/step - loss: 0.2708 - accuracy: 0.9190 - val_loss: 0.2675 - val_accuracy: 0.9204
Epoch 4/5
3824/3824 [==============================] - 7s 2ms/step - loss: 0.2703 - accuracy: 0.9190 - val_loss: 0.2672 - val_accuracy: 0.9204
Epoch 5/5
3824/3824 [==============================] - 6s 2ms/step - loss: 0.2697 - accuracy: 0.9190 - val_loss: 0.2674 - val_accuracy: 0.9204


In [ ]:
model22.evaluate(X_test_scaled,  y_test, verbose=2)

1912/1912 - 2s - loss: 0.2674 - accuracy: 0.9204 - 2s/epoch - 930us/step


[0.26740866899490356, 0.9203948974609375]

### 3. Keras Tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten
import IPython

In [ ]:
def model_builder3(hp):

  model3 = Sequential()

  # Dense layer에서 노드 수를 조정
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)

  model3.add(Dense(units = hp_units, activation='relu'))
  model3.add(Dense(units = hp_units, activation='relu'))

  model3.add(Dense(1, activation='sigmoid'))

  # Optimizer의 학습률(learning rate)을 조정[0.01, 0.001, 0.0001]
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  model3.compile(optimizer=keras.optimizers.Adam(learning_rate = hp_learning_rate), 
                loss=keras.losses.BinaryCrossentropy(from_logits = True), 
                metrics=['accuracy'])

  return model3

In [ ]:
tuner = kt.Hyperband(model_builder3,
                     objective = 'val_accuracy', 
                     max_epochs = 5, 
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(X_train_scaled, y_train, epochs = 1, batch_size=64, validation_data = (X_test_scaled,y_test), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
최적화된 Dense 노드 수 : {best_hps.get('units')} 
최적화된 Learning Rate : {best_hps.get('learning_rate')} 
""")

Trial 10 Complete [00h 01m 22s]
val_accuracy: 0.9204112887382507

Best val_accuracy So Far: 0.9204112887382507
Total elapsed time: 00h 07m 08s
INFO:tensorflow:Oracle triggered exit

최적화된 Dense 노드 수 : 320 
최적화된 Learning Rate : 0.01 



In [ ]:
from tensorflow.keras import regularizers

initializer = tf.keras.initializers.HeNormal()

model33 = Sequential()
model33.add(Dense(best_hps.get('units'), 
                activation='relu', kernel_initializer=initializer,          
                kernel_regularizer=regularizers.l2(0.001),
                activity_regularizer=regularizers.l1(0.001)))
model33.add(Dense(best_hps.get('units'),
                activation='relu', kernel_initializer=initializer,            
                kernel_regularizer=regularizers.l2(0.001),
                activity_regularizer=regularizers.l1(0.001)))
model33.add(Dense(1, activation='sigmoid'))

model33.compile(optimizer=keras.optimizers.Adam(learning_rate = best_hps.get('learning_rate')), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

results = model33.fit(X_train_scaled, y_train, epochs=5, batch_size=64, validation_data=(X_test_scaled,y_test))

Epoch 1/5
3824/3824 [==============================] - 19s 5ms/step - loss: 0.3015 - accuracy: 0.9189 - val_loss: 0.2770 - val_accuracy: 0.9204
Epoch 2/5
3824/3824 [==============================] - 19s 5ms/step - loss: 0.2764 - accuracy: 0.9190 - val_loss: 0.2746 - val_accuracy: 0.9204
Epoch 3/5
3824/3824 [==============================] - 18s 5ms/step - loss: 0.2757 - accuracy: 0.9190 - val_loss: 0.2723 - val_accuracy: 0.9204
Epoch 4/5
3824/3824 [==============================] - 18s 5ms/step - loss: 0.2757 - accuracy: 0.9190 - val_loss: 0.2726 - val_accuracy: 0.9204
Epoch 5/5
3824/3824 [==============================] - 17s 4ms/step - loss: 0.2758 - accuracy: 0.9190 - val_loss: 0.2723 - val_accuracy: 0.9204


In [ ]:
model33.evaluate(X_test_scaled,  y_test, verbose=2)

1912/1912 - 2s - loss: 0.2723 - accuracy: 0.9204 - 2s/epoch - 1ms/step


[0.27232635021209717, 0.9203948974609375]

### 4. Dropout

In [ ]:
initializer = tf.keras.initializers.HeNormal()

model33 = Sequential()
model33.add(Dense(best_hps.get('units'), 
                activation='relu', kernel_initializer=initializer,          
                kernel_regularizer=regularizers.l2(0.001),
                activity_regularizer=regularizers.l1(0.001)))
model.add(Dropout(0.1))
model33.add(Dense(best_hps.get('units'),
                activation='relu', kernel_initializer=initializer,            
                kernel_regularizer=regularizers.l2(0.001),
                activity_regularizer=regularizers.l1(0.001)))
model.add(Dropout(0.1))
model33.add(Dense(1, activation='sigmoid'))

model33.compile(optimizer=keras.optimizers.Adam(learning_rate = best_hps.get('learning_rate')), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

results = model33.fit(X_train_scaled, y_train, epochs=5, batch_size=64, validation_data=(X_test_scaled,y_test))

Epoch 1/5
3824/3824 [==============================] - 19s 5ms/step - loss: 0.3071 - accuracy: 0.9188 - val_loss: 0.2756 - val_accuracy: 0.9204
Epoch 2/5
3824/3824 [==============================] - 18s 5ms/step - loss: 0.2761 - accuracy: 0.9190 - val_loss: 0.2738 - val_accuracy: 0.9204
Epoch 3/5
3824/3824 [==============================] - 18s 5ms/step - loss: 0.2758 - accuracy: 0.9190 - val_loss: 0.2724 - val_accuracy: 0.9204
Epoch 4/5
3824/3824 [==============================] - 17s 5ms/step - loss: 0.2760 - accuracy: 0.9190 - val_loss: 0.2724 - val_accuracy: 0.9204
Epoch 5/5
3824/3824 [==============================] - 18s 5ms/step - loss: 0.2755 - accuracy: 0.9190 - val_loss: 0.2722 - val_accuracy: 0.9204


In [ ]:
model33.evaluate(X_test_scaled,  y_test, verbose=2)

1912/1912 - 2s - loss: 0.2722 - accuracy: 0.9204 - 2s/epoch - 1ms/step


[0.2722399830818176, 0.9203948974609375]